# Project 

## Project Description

Dataset was last updated 1-8-2021.

## Part 1: Data Retrieval & Preprocessing


In [9]:
# Import Libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the JSON data
tweets_data = json.load(open("Datasets/tweets_01-08-2021.json"))

# Convert JSON data to pandas dataframe
tweets_df = pd.json_normalize(tweets_data)

In [10]:
# Drop columns isFlagged, isDeleted, device, favorites, and retweets from the dataframe

tweets_df.drop(columns=['isFlagged', 'isDeleted', 'device', 'favorites', 'retweets', 'id'], inplace=True)

tweets_df.head()

,text,isRetweet,date
0,Republicans and Democrats have both created ou...,f,2011-08-02 18:07:48
1,I was thrilled to be back in the Great city of...,f,2020-03-03 01:34:50
2,RT @CBS_Herridge: READ: Letter to surveillance...,t,2020-01-17 03:22:47
3,The Unsolicited Mail In Ballot Scam is a major...,f,2020-09-12 20:10:58
4,RT @MZHemingway: Very friendly telling of even...,t,2020-01-17 13:13:59


In [11]:
# Remove all rows with less than 15 characters in the text column
tweets_df = tweets_df[(tweets_df['text'].str.len() > 20) & (tweets_df['text'].str.len() < 200)]

# Remove all tweets that are retweets and all tweets that are older than 01-01-2020
tweets_df = tweets_df[tweets_df['isRetweet'] == 'f']
tweets_df['date'] = pd.to_datetime(tweets_df['date'])

print(tweets_df.shape)

(39746, 3)
